In [1]:
import xml.etree.ElementTree as ET
from PIL import Image
import os
import pandas as pd
import numpy as np
import json

In [ ]:
anno_pathes_json = []
file_name = '.json'

absolute_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\BallerTV'
crop_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops'

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

for root, dirs, files in os.walk(absolute_path):    
    for file in files:
        # проверяем, есть ли заданный файл в списке файлов текущей папки
        if (file_name in file) & ('.jsonl' not in file):
            # если есть, то добавляем путь в список
            anno_pathes_json.append(os.path.join(root, file)) 

In [ ]:
alpha = 0.1
num = 0
annos_list = []
anno_dict = {'file_name': [], 'text': []}
pathes_list = []

for anno in tqdm(anno_pathes_json):
    with open(anno, 'r') as f:
        data = json.load(f)

    #Преобразуем json  в dataFrame
    points = pd.read_json(json.dumps(data['shapes']))
    x = pd.read_json(anno, orient='index')
    frame = x.iloc[3, :][0]

    img_path = anno.replace('anno', 'frames')
    img_path = img_path.replace('.json', '.jpeg')
    img = Image.open(img_path)

    for j in range(len(points)):
        out_file = {}
        point_obj = pd.DataFrame(points.iloc[j, 0:2])
        point = point_obj.iloc[1, :]
        label = point_obj.iloc[0, :]
        split_label = label[j].split('_')
        number = split_label[-1]

        for box in point:   

            if number.isdigit():

                xtl, ytl, xbr, ybr = int(box[0][0]), int(box[0][1]), int(box[1][0]), int(box[1][1])
                            
                x_dif = np.abs(xtl - xbr)
                y_dif = np.abs(ytl - ybr)

                xtl = xtl - alpha * (x_dif / 2)
                xbr = xbr + alpha * (x_dif / 2)
                ytl = ytl - alpha * (y_dif / 2)
                ybr = ybr + alpha * (y_dif / 2)

                # Сделаем кроп номера на футболке
                croppped_image = img.crop((xtl, ytl, xbr, ybr))
                full_path = crop_path + '\\' + str(number)                

                if not os.path.exists(full_path):
                    os.makedirs(full_path)    
                    #print(f'Папка {number} создана')  
                                  
                res_path = full_path +'\\' + f'ballerTV_{str(num)}example.jpg'
                num += 1

                annos_list.append(str(number))
                pathes_list.append(res_path )
                anno_dict['text'] = annos_list
                anno_dict['file_name'] = pathes_list
                
                croppped_image.save(res_path)

df = pd.DataFrame(anno_dict, index=None)
df.to_csv("anno_ballerTV.csv")
df.head()

In [ ]:
crop_path = "C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops"
absolute_path = "C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\BallerTV"

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

file_name = 'annotations.xml'

anno_pathses = []
# пройдемся рекурсивно по всем папкам, начиная с absolute_path
for root, dirs, files in os.walk(absolute_path):    
    # проверяем, есть ли заданный файл в списке файлов текущей папки
    if file_name in files:
        # если есть, то добавляем путь в список
        anno_pathses.append(os.path.join(root, file_name))          

In [ ]:
nums = []
num = 0
folder_pathes = []
pathes_list = []
annos_list = []
anno_dict = {'file_name': [], 'text': []}

alpha = 0.2

for path in tqdm(anno_pathses):

    # Загрузим XML файл
    tree = ET.parse(path)

    # Получим корневой элемент XML файла
    root = tree.getroot()

    # Проитерируемся по дочерним элементам корня
    for image in root.findall('image'):
        
        image_id = image.get('id')
        file_name = image.get('name')
        file_name = 'data\\' + file_name
  
        # Переберем все baundingbox`s
        for box in image.findall('box'):
            
            image = Image.open(file_name)
           
            label = box.get('label')

            # Проверим является ли метка меткой номера
            if label == "Jersey":
                xtl = int(float(box.get('xtl')))
                ytl = int(float(box.get('ytl')))
                xbr = int(float(box.get('xbr')))
                ybr = int(float(box.get('ybr')))
                number = box.find('attribute[@name="number"]').text
                if number == '-1':
                    continue
               
                x_dif = np.abs(xtl - xbr)
                y_dif = np.abs(ytl - ybr)

                xtl = xtl - alpha * (x_dif / 2)
                xbr = xbr + alpha * (x_dif / 2)
                ytl = ytl - alpha * (y_dif / 2)
                ybr = ybr + alpha * (y_dif / 2)

                # Сделаем кроп номера на футболке
                croppped_image = image.crop((xtl, ytl, xbr, ybr))
                full_path = crop_path + '\\' + str(number)
                num += 1

                if not os.path.exists(full_path):
                    os.makedirs(full_path)    
                    #print(f'Папка {number} создана')  
                                  
                res_path = full_path +'\\' + f'ballerTV_{str(num)}example.jpg'

                annos_list.append(int(number))
                pathes_list.append(res_path )
                anno_dict['text'] = annos_list
                anno_dict['file_name'] = pathes_list
                
                croppped_image.save(res_path)

df = pd.DataFrame(anno_dict, index=None)
df.to_csv("anno_ballerTV2.csv")
df.head()


In [ ]:
anno_pathes_json = []
file_name = '.json'

absolute_path = 'data\\basketball\\16-11-2022'
crop_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops'

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

for root, dirs, files in os.walk(absolute_path):    
    for file in files:
        # проверяем, есть ли заданный файл в списке файлов текущей папки
        if (file_name in file) & ('.jsonl' not in file):
            # если есть, то добавляем путь в список
            anno_pathes_json.append(os.path.join(root, file)) 

In [ ]:
num = 0
alpha = 0.2
annos_list = []
anno_dict = {'file_name': [], 'text': []}
pathes_list = []

for i, anno in enumerate(anno_pathes_json):
    with open(anno, 'r') as f:
        data = json.load(f)
       

    frames = pd.read_json(json.dumps(data['frames']))
    print(f'{i + 1}ая папка из {len(anno_pathes_json)}')


    for i in tqdm(range(len(frames))):

        img_name = frames['img_name'][i]
        bboxes = frames['detections'][i]
        #print(bboxes)

        anno_path = anno.replace('anno', 'frames')
        anno_path = anno_path.replace('.json', '\\')
        anno_path_fin = anno_path + str(img_name)

        #image = cv2.cvtColor(cv2.imread(anno_path_fin), cv2.COLOR_BGR2RGB)
        img = Image.open(anno_path_fin)

        # Создание объекта ImageDraw для рисования
        #draw = ImageDraw.Draw(image)

        for detection in bboxes:
        
            ocr_jersey = detection['ocr_jersey']
            if ocr_jersey != '-1':
                jersey_bbox_global = detection['jersey_bbox_global']
                xtl = int(jersey_bbox_global[0])
                ytl = int(jersey_bbox_global[1])
                xbr = int(jersey_bbox_global[2])           
                ybr = int(jersey_bbox_global[3])
                
                x_dif = np.abs(xtl - xbr)
                y_dif = np.abs(ytl - ybr)

                xtl = xtl - alpha * (x_dif / 2)
                xbr = xbr + alpha * (x_dif / 2)
                ytl = ytl - alpha * (y_dif / 2)
                ybr = ybr + alpha * (y_dif / 2)

                #draw.rectangle([(jersey_bbox_global[0], jersey_bbox_global[1]), (jersey_bbox_global[2], jersey_bbox_global[3])], outline='red')
                #cv2.rectangle(image, (xtl, ytl), (xbr, ybr), 2)
                croppped_image = img.crop((xtl, ytl, xbr, ybr))
                full_path = crop_path + '\\' + str(ocr_jersey)
                res_path = full_path + '\\' f'basketball_{num}example.jpg'
                num += 1

                if not os.path.exists(full_path):
                        os.makedirs(full_path)    

                annos_list.append(str(ocr_jersey))
                pathes_list.append(res_path )
                anno_dict['text'] = annos_list
                anno_dict['file_name'] = pathes_list

                croppped_image.save(res_path)

df = pd.DataFrame(anno_dict, index=None)
df.to_csv("anno_basketball.csv")
df.head()

In [ ]:
crop_path = "C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops"
absolute_path = "C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\FIBA3x3\\fiba3x3_tracking_135"

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

file_name = 'annotations.xml'

anno_pathses = []
# пройдемся рекурсивно по всем папкам, начиная с absolute_path
for root, dirs, files in os.walk(absolute_path):    
    # проверяем, есть ли заданный файл в списке файлов текущей папки
    if file_name in files:
        # если есть, то добавляем путь в список
        anno_pathses.append(os.path.join(root, file_name))  

In [ ]:
nums = []
num = 0
folder_pathes = []
pathes_list = []
annos_list = []
anno_dict = {'file_name': [], 'text': []}

alpha = 0.2

for path in tqdm(anno_pathses):

    # Загрузим XML файл
    tree = ET.parse(path)

    # Получим корневой элемент XML файла
    root = tree.getroot()

    # Проитерируемся по дочерним элементам корня
    for image in root.findall('image'):
        
        image_id = image.get('id')
        file_name = image.get('name')
        #file_name = file_name.replace('/', '\\')
        file_name = os.path.join(absolute_path, file_name)
        if 'fiba3x3/images/331/' in file_name:
            file_name = file_name.replace('fiba3x3/images/331', 'frames')
        file_name = file_name.replace('frames', 'frames_135')
        # Переберем все baundingbox`s
        for box in image.findall('box'):
            
            image = Image.open(file_name)
           
            label = box.get('label')

            # Проверим является ли метка меткой номера
            if label == "Jersey":
                xtl = int(float(box.get('xtl')))
                ytl = int(float(box.get('ytl')))
                xbr = int(float(box.get('xbr')))
                ybr = int(float(box.get('ybr')))
                number = box.find('attribute[@name="number"]').text

                x_dif = np.abs(xtl - xbr)
                y_dif = np.abs(ytl - ybr)

                xtl = xtl - alpha * (x_dif / 2)
                xbr = xbr + alpha * (x_dif / 2)
                ytl = ytl - alpha * (y_dif / 2)
                ybr = ybr + alpha * (y_dif / 2)

                # Сделаем кроп номера на футболке
                croppped_image = image.crop((xtl, ytl, xbr, ybr))
                full_path = crop_path + '\\' + str(number)
                num += 1

                if not os.path.exists(full_path):
                    os.makedirs(full_path)    
                    #print(f'Папка {number} создана')  
                                  
                res_path = full_path +'\\' + f'fiba_{str(num)}example.jpg'

                annos_list.append(int(number))
                pathes_list.append(res_path )
                anno_dict['text'] = annos_list
                anno_dict['file_name'] = pathes_list
                
                croppped_image.save(res_path)

df = pd.DataFrame(anno_dict, index=None)
df.to_csv("anno_fiba.csv")
df.head()

In [ ]:
anno_pathes_json = []
img_pathes = {}
file_name = '.json'
img_name = '.jpg'

absolute_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\NCAA\\NCAA_1_jersey\\second_task'
crop_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops'
img_folder_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\NCAA\\NCAA_1_tracking\\playerTrackingFrames'

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

for root, dirs, files in os.walk(absolute_path):      
    for file in files:
        # проверяем, есть ли заданный файл в списке файлов текущей папки
        if (file_name in file) & ('.jsonl' not in file):
            # если есть, то добавляем путь в список
            anno_pathes_json.append(os.path.join(root, file)) 

for root, dirs, files in os.walk(img_folder_path):        
    for file in files:
        # проверяем, есть ли заданный файл в списке файлов текущей папки
        if (img_name in file):
            # если есть, то добавляем путь в список
            sh_file = file.replace('frame_', '')
            img_pathes[sh_file] = os.path.join(root, file)  

In [ ]:
alpha = 0.1
num = 0
annos_list = []
anno_dict = {'file_name': [], 'text': []}
pathes_list = []

for anno in tqdm(anno_pathes_json):
    with open(anno, 'r') as f:
        data = json.load(f)

    #Преобразуем json  в dataFrame
    points = pd.read_json(json.dumps(data['shapes']))
    img_name = str(json.dumps(data['imagePath']))
    img_path = img_name.replace('"', '')
    img_path = os.path.split(img_path)
    img_path = img_pathes[img_path[-1]]

    img = Image.open(img_path)
    

    for j in range(len(points)):
        out_file = {}    

        if points["shape_type"][j] == "rectangle":                       
        
            label = points['label']
            split_label = label[j].split('_')
            number = split_label.pop()
            
            point_obj = points['points']
            singl_point = point_obj[j]
            [xtl, ytl], [xbr, ybr] = singl_point

            x_dif = np.abs(xtl - xbr)
            y_dif = np.abs(ytl - ybr)

            xtl = xtl - alpha * (x_dif / 2)
            xbr = xbr + alpha * (x_dif / 2)
            ytl = ytl - alpha * (y_dif / 2)
            ybr = ybr + alpha * (y_dif / 2)

            # Сделаем кроп номера на футболке
            croppped_image = img.crop((xtl, ytl, xbr, ybr))
            full_path = crop_path + '\\' + str(number)                

            if not os.path.exists(full_path):
                os.makedirs(full_path)    
                #print(f'Папка {number} создана')  
                                  
            res_path = full_path +'\\' + f'ncaa_{str(num)}example.jpg'
            num += 1

            annos_list.append(str(number))
            pathes_list.append(res_path )
            anno_dict['text'] = annos_list
            anno_dict['file_name'] = pathes_list
                
            croppped_image.save(res_path)

df = pd.DataFrame(anno_dict, index=None)
df.to_csv("anno_ncaa.csv")
df.head()  